In [1]:
import ujson
import numpy 
import datetime

# Load Data

In [2]:
summaries = {}
ctr = 0
for line in open('user_project_summaries.json'): # lazy iteration because the file is large
    print ctr,
    ctr+=1
    summaries.update(ujson.loads(line))
    if ctr==20:
        break

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19


In [3]:
from features import *

In [7]:
not_tutorials = {}

for i in range(10): 
    print i 
    notT_list = ujson.load(open('tutorial_comparisons/nottutorials_' + str(i) +'.json'))
    for entry in notT_list: 
        if entry[0] not in summaries:
            continue
        proj_name = entry[1].split("_summary.json")[0]
        if entry[0] not in not_tutorials: 
            not_tutorials[entry[0]] = {}
        try: 
            not_tutorials[entry[0]][proj_name] = summaries[entry[0]][proj_name]
        except KeyError: 
            print entry[0], proj_name


0
1
2
3
4
5
6
7
8
9
00999 p005_006_Draw
00999 p002_002_WildLifeShow
00999 p010_010_RememberInformation
00999 p011_011_DetermineLocation
00999 p006_004_Draw2
00999 p012_012_ProcessAList
00999 p008_008_MoleMash


In [8]:
not_tutorials = {user: not_tutorials[user] for user in not_tutorials.keys() if len(not_tutorials[user])>0}
# SR's fix

## Language

Create Language Data Set and Initialize Language Codes

In [9]:
'''We are using all the data from the user from whom we we have previously 
classfied what their primary langauge is '''

isocodes = ujson.load(open('isocodes.json'))  # mapping from  iso code to language name
user_langs = ujson.load(open('user_inferredlangs.json'))  # mapping from userid to inferred language
#remove Latin because it's not a reliable inference
#TODO for later: get next-best language besides Latin in the inference function
# remove uncommon languages
lang_counts = {}
for user in user_langs:
    lang = user_langs[user]
    lang_counts[lang] = lang_counts.get(lang, 0) + 1
        
user_langs = {user: lang for user, lang in user_langs.items() if lang!='la' and lang_counts[lang]>=500 and user in summaries}

langset = sorted(list(set(user_langs.values())))
lang_num_map = {lang: i for i, lang in enumerate(langset)} #key is langauge isocode, value is number 
print 'lang data created with', len(user_langs), 'users and', len(langset), 'languages'

lang data created with 17638 users and 6 languages


## Time

In [10]:
# time utilities
def convert_time(el):
    """Human readable time"""
    el = int(str(el)[:10])
    return datetime.date.fromtimestamp(el)

def timediff(t1, t2):
    """difference between times (millisec precision) as days"""
    return (t1-t2)/(86400.*1000)

In [11]:
earliest = 1e100
latest = 0

ctr = 0
for user in summaries:
    for project in summaries[user]:
        ctime = summaries[user][project]["**created"]
        if ctime < earliest:
            earliest = ctime
        if ctime > latest:
            latest = ctime
            
    ctr+=1
    if ctr%1000==0:
        print ctr/1000,
            
print
print 'Earliest:', convert_time(earliest) 
print 'Latest:', convert_time(latest)
MAXDUR = 300
MINDUR = 150
# get subset of data from "old" users whose earliest creation date is more than MAXDUR days before the end of the dataset
# (giving them a good chance to stay on for more than MINDUR days),
# AND have activity within MAXDUR days if they are active beyond MINDUR (to be fair to recent users)

old_users = set() 

ctr = 0
for user in summaries:
    ctimes = [project["**created"] for project in summaries[user].values()]
    if ctimes==[]:
        print user, 'has no projects'
        continue
    start_time = min(ctimes)
    if timediff(latest, start_time)>MAXDUR:  # first restrict to old users
        # creation times since start date of user
        ctimes_from_start = [timediff(project["**created"], start_time) for project in summaries[user].values()]  
        # projects created at least MINDUR days after start
        post_mindur = [ctime for ctime in ctimes_from_start if ctime>MINDUR]
        # consider users who either were not active after MINDUR days, or had activity between MINDUR and MAXDUR days
        if post_mindur==[] or len(filter(lambda length: length<=MAXDUR, post_mindur))>0:
            old_users.add(user)
            
    ctr+=1
    if ctr%1000==0:
        print ctr/1000,
        
print
print 'Filtered to', len(old_users), 'users from', len(summaries)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Earliest: 2013-03-27
Latest: 2016-03-10
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 00999 has no projects

Filtered to 10926 users from 20000


In [12]:
# print list(old_users)[0]
# summaries['11542'].keys()

In [13]:
def createKeywordDict(projects): 
    '''creates a dictionary of all of the classes a user used in their projects and how many times each class was used'''
    numScreens = getScreenNames(projects)
    keywordDict = {}
    i = 0
    while i < len(projects):
        for screenName in numScreens[i]:
            if screenName in projects[i].keys():
                if  not isinstance(projects[i][screenName]['Components'], unicode):
#                    if 'Number of Components' in projects[i][screenName]['Components'].keys():
                    if not isinstance(projects[i][screenName]['Components']['Type and Frequency'],unicode):
                        for key in projects[i][screenName]['Components']['Type and Frequency']:
                            if key in keywordDict:
                                keywordDict[key] = keywordDict[key] + projects[i][screenName]['Components']['Type and Frequency'][key]
                            else: 
                                keywordDict[key] = projects[i][screenName]['Components']['Type and Frequency'][key]
        i+=1
    
    return keywordDict 

In [14]:
def projectKeywords(project): 
    '''the same as createKeywordDict, but for one project'''
    keywordDict = {}

    screenNames = [key for key in project.keys() if '*' not in key]
    
    for screenName in screenNames:
        if screenName in project.keys():
            if  not isinstance(project[screenName]['Components'], unicode):
#                    if 'Number of Components' in projects[i][screenName]['Components'].keys():
                if not isinstance(project[screenName]['Components']['Type and Frequency'],unicode):
                    for key in project[screenName]['Components']['Type and Frequency']:
                        if key in keywordDict:
                            keywordDict[key] = keywordDict[key] + project[screenName]['Components']['Type and Frequency'][key]
                        else: 
                            keywordDict[key] = project[screenName]['Components']['Type and Frequency'][key]
    
    return keywordDict 

In [15]:
# for user in first_ten[0:1]: 
#     projects= getProjects(user, "LANGUAGE")
#     print createKeywordDict(projects)

In [16]:
def blockClassFrequency(projects): 
    '''a dictionary that contains the number of projects
    a user used each class of blocks in that was used by the user at least once'''
    freqDict = {} 
    numProjects = len(projects)
    for project in projects: 
        projectDict = projectKeywords(project)
        keys = projectDict.keys()
        for key in keys: 
            if key not in freqDict.keys(): 
                freqDict[key] = 1
            else: 
                freqDict[key] +=1
    finalKeys = freqDict.keys() 
    for key in finalKeys: 
        freqDict[key] = float(freqDict[key])
    return freqDict

In [17]:
# for user in first_ten: 
#     print blockClassFrequency(getProjects(user,"LANGUAGE"))

In [18]:
def projectFunctions(project):
    '''returns how many times a user used a function a particular in a project'''
    freqDict = {}
    screenNames = [key for key in project.keys() if '*' not in key]
    i = 0
    while i < len(projects):
        for screenName in screenNames:
            if screenName in project.keys():
                if  not isinstance(project[screenName]['Blocks'], unicode):
                    if  not isinstance(project[screenName]['Blocks']['Active Blocks'],unicode):
                        if project[screenName]['Blocks']['Active Blocks'] != 'NO ACTIVE BLOCKS':
                            all_funcs =  project[screenName]['Blocks']['Active Blocks']['Types'].keys()                                               
                            for func in all_funcs:
                                sfunc = []
                                if "_" in func: 
                                    sfunc = func.split('_')
                                if "." in func:
                                    sfunc = func.split('.')
                                if type(sfunc) == list and len(sfunc) > 1:
                                    key = sfunc[1]
                                    if func[1] not in freqDict.keys(): 
                                         freqDict[key] =  project[screenName]['Blocks']['Active Blocks']['Types'][func]
                                    else: 
                                        freqDict[key] += project[screenName]['Blocks']['Active Blocks']['Types'][func] 
        i+=1
                            
    return freqDict

In [19]:
def blockFuncFrequency(projects):
    '''a dictionary that contains the number of projects
    a user used each function. each function must have been used by the user at least once'''
    freqDict = {} 
    numProjects = len(projects)
    for project in projects: 
        projectDict = projectFunctions(project)
        keys = projectDict.keys()
        for key in keys: 
            if key not in freqDict: 
                freqDict[key] = 1
            else: 
                freqDict[key] +=1
    finalKeys = freqDict.keys() 
    for key in finalKeys: 
        freqDict[key] = float(freqDict[key])
    return freqDict

In [20]:
def functionDict(projects):
    '''returns a dictionary how many times a user used each function (that was used at least once) throughout her projects. function equivalent to createKeywordDict'''
    freqDict = {}
    screenNames = getScreenNames(projects)
    i = 0
    while i < len(projects):
        for screenName in screenNames[i]:
            if screenName in project: #removed .keys() 
                if  not isinstance(projects[i][screenName]['Blocks'], unicode):
                    if  not isinstance(projects[i][screenName]['Blocks']['Active Blocks'],unicode):
                        if projects[i][screenName]['Blocks']['Active Blocks'] != 'NO ACTIVE BLOCKS':
                            all_funcs =  projects[i][screenName]['Blocks']['Active Blocks']['Types'].keys()                                               
                            for func in all_funcs:
                                sfunc = []
                                if "_" in func: 
                                    sfunc = func.split('_')
                                if "." in func:
                                    sfunc = func.split('.')
                                if type(sfunc) == list and len(sfunc) > 1:
                                    key = sfunc[1]
                                    if func[1] not in freqDict.keys(): 
                                         freqDict[key] =  projects[i][screenName]['Blocks']['Active Blocks']['Types'][func]
                                    else: 
                                        freqDict[key] += projects[i][screenName]['Blocks']['Active Blocks']['Types'][func] 
        i+=1
                            
    return freqDict

In [21]:
# '''this code compiles the frequency of all classes among all users. The frequency dictionary of all classes is allKeysD. 
# commonList holds a list of all classes that are used more than 15 times across all users. rareList holds a list of all class that are used fewer than 15 times.'''
# allKeys = set()
# allKeysD = {}
# i=0 
# for user in all_users: 
#     keywords = createKeywordDict(getProjects(user, "LANGUAGE")).keys()
#     for key in keywords: 
#         if key not in allKeys: 
#             allKeys.add(key)
#             allKeysD[key] = 1
#         else: 
#             allKeysD[key] +=1

#     if i % 1000 == 0: 
#         print i
#     i+=1 
# print allKeysD, '\n\n'


In [22]:
# commonList =[]
# rareList = []
# for k in allKeys: 
#     if allKeysD[k] > 15: 
#         commonList.append((k, allKeysD[k]))
#     else:
#         rareList.append((k, allKeysD[k])) 


In [23]:
# print sorted(commonList, key= lambda x: x[1])

In [24]:
# print sorted(rareList, key= lambda x: x[1])

All possible classes[u'Web', u'Canvas', u'FtcIrSeekerSensor', u'BluetoothServer', u'ActivityStarter', u'Texting', u'SoundRecorder', u'HorizontalArrangement', u'Sharing', u'FloatingButton', u'Sound', u'LinkedData', u'SpeechRecognizer', u'mrSoundMeter', u'FtcOpMode', u'SensorDB', u'NxtDrive', u'LinkedDataForm', u'mrPanel', u'Voting', u'mrButton', u'ProximitySensor', u'InAppBilling', u'Pedometer', u'NxtColorSensor', u'LightSensor', u'NearField', u'YandexTranslate', u'mrSoundRecorder', u'TableArrangement', u'GalleryViewer', u'CheckBox', u'Clock', u'FtcRobotController', u'Label', u'Camera', u'EmailPicker', u'AdMobInterstitial', u'NxtTouchSensor', u'ContactPicker', u'FusiontablesControl', u'VSVArrangement', u'PhoneNumberPicker', u'Button', u'NxtDirectCommands', u'ListPicker', u'MIOIOUltrasonidos', u'AdAmazonInterstitial', u'PhoneStatus', u'BarcodeScanner', u'DigitalRead', u'BLE', u'FtcServo', u'GoogleDrive', u'Camcorder', u'Notifier', u'TextBox', u'MobFoxInterstitial', u'Image', u'KitchenSink', u'TinyWebDB', u'GameClient', u'UsbAccessory', u'Sharer', u'Spinner', u'BluetoothClient', u'NxtLightSensor', u'LocationSensor', u'TextToSpeech', u'OrientationSensor', u'FtcGamepad', u'mrSlider', u'ListView', u'FtcOpticalDistanceSensor', u'MMedia', u'WebViewer', u'MMediaInterstitial', u'mrCanvas', u'FtcDcMotor', u'GyroscopeSensor', u'AdAmazon', u'Twitter', u'iSENSE', u'FtcDeviceInterfaceModule', u'VideoPlayer', u'AdMob', u'TimePicker', u'HSVArrangement', u'LinkedDataListPicker', u'FtcTouchSensor', u'mrSound', u'DatePicker', u'VerticalArrangement', u'XYChart', u'FirebaseDB', u'NxtUltrasonicSensor', u'SQLite', u'GoogleMap', u'Slider', u'PasswordTextBox', u'ImagePicker', u'Dropbox', u'WICEDSense', u'TinyDB', u'mrSphero', u'GoogleCloudMessaging', u'mrPlayer', u'Chronometer', u'DropBox', u'AccelerometerSensor', u'PhoneCall', u'Player', u'File', u'NxtSoundSensor', u'UdooArduino

In [25]:
# all_users = summaries.keys()

In [26]:
# first_thousand = all_users[0:1000]
# first_ten = all_users[0:10]

In [27]:
# #list of all functions used
# allFunctions = set()
# allFunctionsD = {}
# i=0 
# for user in all_users: 
#     functions = functionDict(getProjects(user, "LANGUAGE")).keys()
#     for funcName in functions
#         if funcName not in allFunctions: 
#             allFunctions.add(funcName)
#             allFunctionsD[funcName] = 1
#         else: 
#             allFunctionsD[funcName] +=1

#     if i % 1000 == 0: 
#         print i
#     i+=1 
# print allFunctionsD, '\n\n'
# commonFList =[]
# rareFList = []
# for k in allFunctionsD: 
#     if allFunctionsD[k] > 15: 
#         commonFList.append(k)
#     else:
#         rareFList.append(k) 


In [28]:
# for k in allFunctionsD: 
#     if allFunctionsD[k] > 15: 
#         commonFList.append((k, allFunctionsD[k]))
#     else:
#         rareFList.append((k, allFunctionsD[k])) 

# #print sorted(commonFList, key= lambda y: y[1]), '\n'
# #print sorted(rareFList, key= lambda z: z[1])

In [29]:
def get_tf(term,project, freqType): 
    numactiveblocks = numActiveBlocks(project) 
    if numactiveblocks == 0: return 0  
    
    if freqType == 'functions': 
        projectFrequency = projectFunctions(project)
    if freqType == 'classes': 
        projectFrequency = projectKeywords(project)
    
    if term in projectFrequency: 
        return projectFrequency[term] #/ float(numactiveblocks)
    else: return 0

In [30]:
import math
def get_idf(term, projects, freqType): 
    '''idf is log(total number of docs/ number of docs with term in it)'''
    
    if freqType == 'functions': 
        projectFrequency = blockFuncFrequency(projects)
    if freqType == 'classes': 
        projectFrequency = blockClassFrequency(projects)
    
    l = float(numProjects(projects))
    if term not in projectFrequency: return 0 
    
    f = projectFrequency[term]
   
    if l == 0 or f == 0: return 0
    
    return math.log(l / float(f))

In [31]:
def get_tfidf(term, projects, freqType):
    idf = get_idf(term, projects, freqType)
    all_tf = 0
    for project in projects:
        tf = get_tf(term, project, freqType)
        all_tf += tf
    return (tf/numProjects(projects)) * idf

In [32]:
def numActiveBlocks(project):
    screenNames = [key for key in project.keys() if '*' not in key]
    totalBlocks = 0
    for screenName in screenNames:
        if screenName in project.keys():
            if  not isinstance(project[screenName]['Blocks'], unicode):
                if  not isinstance(project[screenName]['Blocks']['Active Blocks'],unicode):
                    totalBlocks += project[screenName]['Blocks']['Active Blocks']['*Number of Blocks']
    return totalBlocks

In [33]:
def getScreenNames(projects): 
    screenNames = []
    for project in projects: 
        projectScreens = [key for key in project.keys() if '*' not in key]
        screenNames.append(projectScreens)
    return screenNames

# Features

In [34]:
from features import *
import features
reload(features)
import sys, os
sys.path.append(os.getcwd())

In [35]:
def deltaDeciles(decileDict):
    values = sortDeciles(decileDict)
    deltas = {}
    for i in range(10): 
        if i== 0: pass
        else: deltas[i] = values[i]-values[i-1]
    return deltas


def sortDeciles(decileDict):
    deciles_sorted = []
    keys = decileDict.keys()
    for key in keys: 
        deciles_sorted.insert(int(key[-1]), decileDict[key])
    return deciles_sorted



## Time Features

Get moments from list of project lengths and intervals.

In [36]:
def projectLengthFeatures(projects):
    """moments of project lengths and intervals, as well as number of projects"""
    lengths = getProjectLengths(projects)
    intervals = getProjectIntervals(projects)
    
    #note: feature names are for our reference only
    userDict = {} 
    userDict["mean of lengths"] = numpy.mean(lengths)
    userDict["stddev of lengths"] = numpy.std(lengths)
    userDict["mean of intervals"] = numpy.mean(lengths)
    userDict["stddev of intervals"] = numpy.std(lengths)
    
    userDict["num projects"] = numProjects(projects)
    
    return userDict

Get weekday distibution features.

In [37]:
def dayAnalysisFeatures(projects):
    """number of projects on each day of the week, and the percentage of them on a weekday"""
    byday = numOnDay(projects)

    userDict = {day: byday[i] for i, day in enumerate(["Monday", 
                                                       "Tuesday", 
                                                       "Wednesday", 
                                                       "Thursday", 
                                                       "Friday", 
                                                       "Saturday", 
                                                       "Sunday"])}
    return userDict

Histogram of user's projects in the period.

In [38]:
def decileProjects(projects):
    numbins = 10
    hist = projectsPerUserPeriod(projects, bins=numbins)
    
    userDict = {'decile '+str(i+1): hist[i] for i in range(numbins)}
    return userDict

Here is a function to combine different feature groups.

In [39]:
def combine_featfuncs(funclist):
    def combined(user):
        basedict = funclist[0](user)
        for f in funclist[1:]:
            basedict.update(f(user))
        return basedict
    return combined

## Code Features


In [40]:
def summaryOBlockDecile(projects): 
    hist = decileOrphanBlocks(projects)
    userDict = {'Orphan Decile '+str(i+1): hist[i] for i in range(10)}
    return userDict

In [41]:
def summaryDecileTLBlocks(projects): 
    hist = decileTypesTopLevelBlocks(projects)
    userDict = {'TopLevel Decile '+str(i+1): hist[i] for i in range(10)}
    return userDict

In [42]:
def summaryDecileNumScreens(projects): 
    hist = decileNumScreens(projects)
    userDict = {'NumScreens Decile '+str(i+1): hist[i] for i in range(10)}
    return userDict

In [43]:
def summaryDeltasTLBlocks(projects): 
    tlblocks = summaryOBlockDecile(projects)
    return deltaDeciles(tlblocks)

In [44]:
def summaryDeltasOBlock(projects): 
    oblock = summaryOBlockDecile(projects)
    return deltaDeciles(oblock)

In [45]:
def summaryDeltasNumScreens(projects): 
    numscreens = summaryOBlockDecile(projects)
    return deltaDeciles(numscreens)

In [46]:
def summaryAverages(projects):
    userDict = {} 
    userDict["mean of numScreens"] = numpy.mean(getNumScreens(projects))
    userDict["NB"] = averageNumBlocks(projects)
    userDict["OB"] = getAverageOrphanBlocks(projects)
    
    userDict["TL"] = getAverageTypeTLBlocks(projects)
    userDict["TL2"] = getAverageNumTLBlocks(projects)
    
    userDict["NC"] = averageNumComponents(projects)
    userDict["NTC"] = averageNumTypeComponents(projects)

    userDict["MC"] = aveNumMediaAssets(projects)
    
    userDict["NP"] = averageNumProcedures(projects)
    userDict["NS"] = averageNumStrings(projects) 
    
    #userDict["KW"] = countKeywords(projects)
    
    varList = getAllVariables(projects)
    userDict["local vars"] = varList[0]
    userDict["global vars"] = varList[1]
    
    return userDict

In [47]:
def funcs_tfidf(projects):
    userDict = {}
    userDict["Click"] = get_tfidf("Click", projects, "classes")
    userDict["variable"] = get_tfidf("variable", projects, "classes")
    userDict["SetText"] = get_tfidf("SetText", projects, "classes")
    userDict["number"] = get_tfidf("number", projects, "classes")
    userDict["GetText"] = get_tfidf("GetText", projects, "classes")
    userDict["if"] = get_tfidf("if", projects, "classes")
    userDict["declaration"] = get_tfidf("declaration", projects, "classes")
    userDict["compare"] = get_tfidf("compare", projects, "classes")
    userDict["add"] = get_tfidf("add", projects, "classes")
    userDict["join"] = get_tfidf("join", projects, "classes")

    return userDict

In [48]:
def classes_tfidf(projects): 
    userDict = {}
    userDict["Button"] = get_tfidf("Button", projects, "functions")
    userDict["Label"] = get_tfidf("Label", projects, "functions")
    userDict["HorizontalArrangement"] = get_tfidf("HorizontalArrangement", projects, "functions")
    userDict["Canvas"] = get_tfidf("Canvas", projects, "functions")
    userDict["TextBox"] = get_tfidf("TextBox", projects, "functions")
    userDict["Clock"] = get_tfidf("Clock", projects, "functions")
    userDict["Sound"] = get_tfidf("Sound", projects, "functions")
    userDict["image"] = get_tfidf("Notifier", projects, "functions")
    userDict["VerticalArrangement"] = get_tfidf("VerticalArrangement", projects, "functions")

    return userDict

# Classification

Get list of projects for each user.

In [49]:
def getAllProjects(userID, no_tutorials): 
    """list of projects sorted by creation times"""
    if (no_tutorials):
        try: 
            projectlist = not_tutorials[userID].values()
        except:
            print userID
    else: 
        projectlist = summaries[userID].values()
    return sorted(projectlist,
                  key=lambda project: project['**created']) 

def projectsInMindur(projectlist):
    """filter projects within user's MINDUR duration from earliest"""
    earliest = projectlist[0]['**created']
    filtered_projects = [projectlist[0]]
    for project in projectlist[1:]:
        if timediff(project['**created'], earliest)>120:#1/29 manually changed from MINDUR to 120
            break
        filtered_projects.append(project)
    return filtered_projects

def userDuration(projectlist):
    """get the duration (difference between last and earliest creation dates)"""
    return timediff(projectlist[-1]['**created'], projectlist[0]['**created'])

def getProjects(userID, task, no_tutorials):
    projectlist = getAllProjects(userID, no_tutorials)
    if task == 'LANGUAGE': 
        return projectlist
    if task == 'RETENTION':
        return projectsInMindur(projectlist)

Use the appropriate set of users for each task with labels.

In [50]:
def get_user_labels(task, not_tutorials):
    if task == 'RETENTION': 
        return {user: int(userDuration(getAllProjects(user, not_tutorials))>MINDUR) for user in old_users} 
    if task == 'LANGUAGE':
        return {user: lang_num_map[lang] for user, lang in user_langs.items()}

### Featurizing

In [61]:
# featurizer functions
time_combined = combine_featfuncs([projectLengthFeatures, dayAnalysisFeatures, decileProjects])  
code_combined = combine_featfuncs([summaryDecileNumScreens,summaryOBlockDecile,summaryDecileTLBlocks])#,summaryAverages,summaryDeltasNumScreens,summaryDeltasOBlock, summaryDeltasOBlock]) #funcs_tfidf, classes_tfidf]) 

In [62]:
from sklearn.feature_extraction import DictVectorizer

all_features = []
time_features = [] 
code_features = []

TASK = 'LANGUAGE'  # toggle between LANGUAGE and RETENTION
userlabels = get_user_labels(TASK, False)

y = []
ctr = 0
for user in userlabels:
    projects = getProjects(user, TASK, False)  # this returns projects sorted by creation date that include tutorials 
    
    y.append(userlabels[user])
    
    time_features.append(time_combined(projects))
    code_features.append(code_combined(projects))
    
    
    # merge time and code feature dicts
    all_features_user = time_features[-1].copy()
    all_features_user.update(code_features[-1])
    all_features.append(all_features_user)
  
    ctr+=1
    if ctr%1000==0:
        print ctr/1000
        


y = numpy.array(y)

timevec = DictVectorizer()

Xtime = timevec.fit_transform(time_features) 

codevec = DictVectorizer()

Xcode = codevec.fit_transform(code_features)

allvec = DictVectorizer()

Xall = allvec.fit_transform(all_features)


. . . . . . . . . . . . . . . . .


In [71]:
print lang_num_map
print y[:10]
ctr = 0
for user in userlabels:
    print user, userlabels[user]
    ctr+=1
    if ctr==10:
        break
    

{u'en': 0, u'zh': 5, u'pt': 4, u'ko': 3, u'it': 2, u'es': 1}
[5 3 5 0 0 1 5 3 5 0]
07539 5
16079 3
11542 5
11543 0
11540 0
11541 1
11547 5
11544 3
11545 5
11548 0


## Tutorial Filtered Featurizing 

In [74]:

#version w/o tutorials
all_features2 = []
time_features2 = [] 
code_features2 = []

TASK = 'LANGUAGE'  # toggle between LANGUAGE and RETENTION
userlabels = get_user_labels(TASK, True)

extras = [] 

y = []
ctr = 0
for user in userlabels:
    if user in not_tutorials: 
        no_t_projects = getProjects(user, TASK, True) #does not include tutorials 

        y.append(userlabels[user])

        time_features2.append(time_combined(no_t_projects))
        code_features2.append(code_combined(no_t_projects))   

        # merge time and code feature dicts

        all_features_user2 = time_features2[-1].copy()
        all_features_user2.update(code_features2[-1])
        all_features2.append(all_features_user2)

        ctr+=1
        if ctr%1000==0:
            print '.',
        
    else: 
         extras.append(user)

print len(extras)            
            
y = numpy.array(y)

timevec2 = DictVectorizer()

Xtime2 = timevec2.fit_transform(time_features2) 

codevec2 = DictVectorizer()

Xcode2 = codevec2.fit_transform(code_features2)

allvec2 = DictVectorizer()

Xall2 = allvec2.fit_transform(all_features2)

. . . . . . . . . . . . . . . . . 68


Scale and standardize features

In [73]:
from sklearn.preprocessing import scale
from numpy import *
# # convert to dense matrices since these are dense anyway
Xtime = scale(Xtime.toarray())
Xtime2 = scale(Xtime2.toarray())

Xcode = scale(Xcode.toarray())
Xcode2 = scale(Xcode2.toarray())

Xall = scale(Xall.toarray())
Xall2 = scale(Xall2.toarray())

print len(y)

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

### Model and Prediction

In [67]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.metrics import f1_score 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix


print 'Chance is', max(numpy.bincount(y))/float(len(y))

foldindices = StratifiedKFold(y)

if TASK=='LANGUAGE':
    labels = map(lambda x:x[0], sorted(lang_num_map.items(), key=lambda x:x[1]))  
    # languages corresponding to class labels as a list
elif TASK=='RETENTION':
    labels = ['left', 'survived']
    
k = 70 #num neighbors for kNN
if TASK == 'RETENTION':
    classifiers = [('knn', KNeighborsClassifier(n_neighbors=k)), 
                   ('logreg', LogisticRegression())]
else:
    classifiers = [('logreg', LogisticRegression(class_weight='balanced', 
                                                             multi_class='multinomial', 
                                                             solver='lbfgs'))]  # no knn because it's too slow

for modelname, model in classifiers:
    for featname, X in [('time', Xtime), ('code', Xcode), ('all', Xall),('time2', Xtime2), ('code2', Xcode2), ('all2', Xall2)]:
        print  'Building a', modelname, 'model with', X.shape[1], featname, 'features for', TASK
    
        cvaccs = numpy.zeros(len(foldindices))
        cvf1 = numpy.zeros(len(foldindices))
        for i, (trainidx, testidx) in enumerate(foldindices):
            print 'Fold', i+1
        
            ytrain = y[trainidx]
            ytest = y[testidx]
        
            Xtrain = X[trainidx, :]  
            Xtest = X[testidx, :]
            
            model.fit(Xtrain, ytrain)
            cvaccs[i] = model.score(Xtest, ytest)
        
            predictions = model.predict(Xtest)
            #cvf1[i] = f1_score(ytest, predictions)
            
            #print '****', modelname, cvaccs[i]
        #analyze last fold only
        cnf_matrix = confusion_matrix(ytest, predictions)  
        #plot_confusion_matrix(cnf_matrix, labels)
        #plt.show()
        
        print 'Average accuracy', numpy.mean(cvaccs)
        print 'Average F1 score', numpy.mean(cvf1)

Chance is 0.513545816733
Building a logreg model with 22 time features for LANGUAGE
Fold 1
Fold 2
Fold 3
Average accuracy 0.0981204650791
Average F1 score 0.0
Building a logreg model with 30 code features for LANGUAGE
Fold 1
Fold 2
Fold 3
Average accuracy 0.105292314614
Average F1 score 0.0
Building a logreg model with 52 all features for LANGUAGE
Fold 1
Fold 2
Fold 3
Average accuracy 0.114398470996
Average F1 score 0.0
Building a logreg model with 22 time2 features for LANGUAGE
Fold 1
Fold 2
Fold 3
Average accuracy 0.36374522772
Average F1 score 0.0
Building a logreg model with 30 code2 features for LANGUAGE
Fold 1
Fold 2
Fold 3
Average accuracy 0.255209125771
Average F1 score 0.0
Building a logreg model with 52 all2 features for LANGUAGE
Fold 1
Fold 2
Fold 3
Average accuracy 0.342117157255
Average F1 score 0.0
